In [1]:
import os
import pandas as pd


# Change your Directory accordingly
os.chdir("D:/Nova/T3/Machine_Learning/Group_Asssignment/Air-Quality-Forecast")

In [2]:
from src.Dataset_Downloader import DatasetDownloader  

dataset_slug = 'rupakroy/lstm-datasets-multivariate-univariate'
download_path = './data/lstm-datasets' 

# Now you can call the function
dataset = DatasetDownloader(dataset_slug, download_path)
dataset.download()


Dataset already exists. Skipping download.


In [3]:
df_pollution, df_pollution_test = dataset.importer()

Datasets imported successfully!


In [4]:
df_pollution.head()

,date,pollution,dew,temp,press,wnd_dir,wnd_spd,snow,rain
0,2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,SE,1.79,0,0
1,2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,SE,2.68,0,0
2,2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,SE,3.57,0,0
3,2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,SE,5.36,1,0
4,2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,SE,6.25,2,0


In [5]:
df_pollution_test.head()

,dew,temp,press,wnd_dir,wnd_spd,snow,rain,pollution
0,-16,4,1027,SE,3.58,0,0,128
1,-17,5,1027,SE,7.60,0,0,77
2,-16,4,1027,SE,9.39,0,0,65
3,-16,1,1028,cv,0.89,0,0,79
4,-14,0,1028,NE,1.79,0,0,93
